Импорт необходимых библиотек и скачивание файла

In [105]:
import pandas as pd
import numpy as np
np.random.seed(42)

In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
csv_file_path = '/content/drive/MyDrive/data_grants_114_v20240218.csv'

In [108]:
data = pd.read_csv(csv_file_path, sep=';')

<ipython-input-108-39a4a1e4a2f4>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv_file_path, sep=';')


Берем только часть данных, иначе они занимают слишком много памяти

In [109]:
data=data.sample(10000)

In [110]:
data=data.reset_index()

Создаем новую фичу - время, которые люди планируют работать над проектом = (предполагаемое время начала проекта - время подачи заявки на грант)

In [111]:
import datetime
work_days = []
for i in range(data.shape[0]):
  date1 = datetime.datetime.strptime(data['date_req'][i],'%Y-%m-%d')
  date2 = datetime.datetime.strptime(data['implem_end'][i],'%Y-%m-%d')
  work_days.append(int((date2-date1).total_seconds()//86400))
data = data.assign(work_days=work_days)

Удаление ненужных колонок

In [112]:
columns_to_drop = ['link','ogrn', 'okato', 'oktmo', 'req_num', 'implem_end', 'contacts', 'rate', 'date_req', 'implem_start', 'org', 'pj_geo', 'cofunding', 'total_money', 'inn', 'index']
for i in columns_to_drop:
  del data[i]

In [113]:
for i in data.columns:
  print(i, data[i].nunique())

name 9753
contest 17
year 8
direction 14
region 87
winner 2
money_req_grant 9549
description 9975
goal 9909
tasks 9933
soc_signif 9957
target_groups 8365
work_days 305


Т.к. в колонке direction всего 14 различных значений, мы можем выполнить one-hot encoding

In [114]:
data=pd.get_dummies(data, columns=["direction"], drop_first=True)

In [117]:
data

Index(['name', 'contest', 'year', 'region', 'winner', 'money_req_grant',
       'description', 'goal', 'tasks', 'soc_signif', 'target_groups',
       'work_days',
       'direction_Защита прав и свобод человека и гражданина, в том числе защита прав заключенных',
       'direction_Охрана здоровья граждан, пропаганда здорового образа жизни',
       'direction_Охрана окружающей среды и защита животных',
       'direction_Поддержка молодежных проектов, реализация которых охватывает виды деятельности, предусмотренные статьей 31 Федерального закона от 12 января 1996 г. № 7-ФЗ «О некоммерческих организациях»',
       'direction_Поддержка молодежных проектов, реализация которых охватывает виды деятельности, предусмотренные статьей 31.1 Федерального закона от 12 января 1996 г. № 7-ФЗ «О некоммерческих организациях»',
       'direction_Поддержка проектов в области культуры и искусства',
       'direction_Поддержка проектов в области науки, образования, просвещения',
       'direction_Поддержка с

In [56]:
data=data.replace(False, 0)
data=data.replace(True, 1)

Импортировали предтренированную модель, которая показывает уровень формальности тестов

In [57]:
from joblib import load
form_model = load('model.joblib')

In [58]:
data=data.reset_index()

In [59]:
form_desc=[]
form_soc=[]
form_task=[]
form_goal=[]
for i in range(10000):
  form_desc.append(form_model.predict_proba([data["description"][i]])[0][1])
  form_soc.append(form_model.predict_proba([data["soc_signif"][i]])[0][1])
  form_task.append(form_model.predict_proba([data["tasks"][i]])[0][1])
  form_goal.append(form_model.predict_proba([data["goal"][i]])[0][1])

In [60]:
data["form_desc"]=form_desc
data["form_soc"]=form_soc
data["form_task"]=form_task
data["form_goal"]=form_goal
del form_desc
del form_soc
del form_task
del form_goal

In [61]:
numeric=['winner', 'money_req_grant', 'work_days',
       'direction_Защита прав и свобод человека и гражданина, в том числе защита прав заключенных',
       'direction_Охрана здоровья граждан, пропаганда здорового образа жизни',
       'direction_Охрана окружающей среды и защита животных',
       'direction_Поддержка молодежных проектов, реализация которых охватывает виды деятельности, предусмотренные статьей 31 Федерального закона от 12 января 1996 г. № 7-ФЗ «О некоммерческих организациях»',
       'direction_Поддержка молодежных проектов, реализация которых охватывает виды деятельности, предусмотренные статьей 31.1 Федерального закона от 12 января 1996 г. № 7-ФЗ «О некоммерческих организациях»',
       'direction_Поддержка проектов в области культуры и искусства',
       'direction_Поддержка проектов в области науки, образования, просвещения',
       'direction_Поддержка семьи, материнства, отцовства и детства',
       'direction_Развитие институтов гражданского общества',
       'direction_Развитие общественной дипломатии и поддержка соотечественников',
       'direction_Сохранение исторической памяти',
       'direction_Социальное обслуживание, социальная поддержка и защита граждан',
       'direction_Укрепление межнационального и межрелигиозного согласия',
       'form_desc', 'form_soc', 'form_task', 'form_goal']

In [62]:
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        try:
            text = text.replace(punctuation, '')
        except:
            print("Err", text)
    return text
data["description"] = data['description'].apply(remove_punctuations)
data["goal"] = data['goal'].apply(remove_punctuations)
data["soc_signif"] = data['soc_signif'].apply(remove_punctuations)
data["target_groups"] = data['target_groups'].apply(remove_punctuations)
data['name'] = data['name'].apply(remove_punctuations)

In [63]:
y=data["winner"]
del data["winner"]

In [64]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
desc_apply=data.description.apply(gensim.utils.simple_preprocess)
soc_apply=data.soc_signif.apply(gensim.utils.simple_preprocess)
task_apply=data.tasks.apply(gensim.utils.simple_preprocess)
goal_apply=data.goal.apply(gensim.utils.simple_preprocess)
# model=gensim.models.Word2Vec(window=5, min_count=2, workers=4, sg=0)
# model.build_vocab(response_new, progress_per=1000)
# model.train(response_new, total_examples=model.corpus_count, epochs=model.epochs)

Также добавили перемнные для длин текстов(в словах)

In [65]:
desc_len=[]
soc_len=[]
dir_len=[]
task_len=[]
goal_len=[]
for i in range(10000):
  desc_len.append(len(desc_apply[i]))
  soc_len.append(len(soc_apply[i]))
  task_len.append(len(task_apply[i]))
  goal_len.append(len(goal_apply[i]))

In [66]:
data["desc_len"]=desc_len
data["soc_len"]=soc_len
data["task_len"]=task_len
data["goal_len"]=goal_len
del desc_len
del soc_len
del task_len
del goal_len

In [68]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Векторизация слов

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
X_train, X_test, y_train, y_test = train_test_split(data, y, random_state=0, train_size = .75, stratify=y)

vc5 = TfidfVectorizer(max_features=4000, stop_words=stop_words)
description_text = vc5.fit_transform(X_train['description'])
test_description_text = vc5.transform(X_test['description'])

vc6 = TfidfVectorizer(max_features=4000, stop_words=stop_words)
goal_text = vc6.fit_transform(X_train['goal'])
test_goal_text = vc6.transform(X_test['goal'])

vc7 = TfidfVectorizer(max_features=4000, stop_words=stop_words)
tasks_text = vc7.fit_transform(X_train['tasks'])
test_tasks_text = vc7.transform(X_test['tasks'])

vc8 = TfidfVectorizer(max_features=4000, stop_words=stop_words)
soc_signif = vc8.fit_transform(X_train['soc_signif'])
test_soc_signif = vc8.transform(X_test['soc_signif'])

del X_train["description"]
del X_test["description"]
del X_train["goal"]
del X_test["goal"]
del X_train["tasks"]
del X_test["tasks"]
del X_train["soc_signif"]
del X_test["soc_signif"]

In [95]:
from joblib import dump
dump(vc8, 'vc_soc.joblib')

['vc_soc.joblib']

In [77]:
features_description = [f'text45_{i}' for i in range(description_text.shape[1])]
test_features_description = [f'text45_{i}' for i in range(test_description_text.shape[1])]


features_goal = [f'text5_{i}' for i in range(goal_text.shape[1])]
test_features_goal = [f'text5_{i}' for i in range(test_goal_text.shape[1])]


features_tasks = [f'text6_{i}' for i in range(tasks_text.shape[1])]
test_features_tasks = [f'text6_{i}' for i in range(test_tasks_text.shape[1])]


features_soc_signif = [f'text654_{i}' for i in range(soc_signif.shape[1])]
test_features_soc_signif = [f'text654_{i}' for i in range(test_soc_signif.shape[1])]

In [78]:
description_text = pd.DataFrame(description_text.toarray(), columns = features_description)
test_description_text = pd.DataFrame(test_description_text.toarray(), columns = test_features_description)


goal_text = pd.DataFrame(goal_text.toarray(), columns = features_goal)
test_goal_text = pd.DataFrame(test_goal_text.toarray(), columns = test_features_goal)


tasks_text = pd.DataFrame(tasks_text.toarray(), columns = features_tasks)
test_tasks_text = pd.DataFrame(test_tasks_text.toarray(), columns = test_features_tasks)


soc_signif = pd.DataFrame(soc_signif.toarray(), columns = features_soc_signif)
test_soc_signif = pd.DataFrame(test_soc_signif.toarray(), columns = test_features_soc_signif)


In [79]:
X_train.reset_index(drop = True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

In [80]:
X_train = pd.concat([X_train[['money_req_grant','form_desc', 'form_soc',
       'form_task', 'form_goal', 'desc_len', 'soc_len',
       'task_len', 'goal_len', 'work_days', 'year']], goal_text, tasks_text, soc_signif, description_text], axis='columns')

X_test = pd.concat([X_test[['money_req_grant','form_desc', 'form_soc',
        'form_task', 'form_goal', 'desc_len', 'soc_len',
       'task_len', 'goal_len', 'work_days', 'year']], test_goal_text, test_tasks_text, test_soc_signif, test_description_text], axis='columns')
X_train_goal=pd.concat([X_train[['form_goal', 'goal_len']], goal_text], axis='columns')
X_test_goal=pd.concat([X_test[['form_goal', 'goal_len']], test_goal_text], axis='columns')

X_train_tasks=pd.concat([X_train[['form_task', 'task_len']], tasks_text], axis='columns')
X_test_tasks=pd.concat([X_test[['form_task', 'task_len']], test_tasks_text], axis='columns')

X_train_soc=pd.concat([X_train[['form_soc', 'soc_len']], soc_signif], axis='columns')
X_test_soc=pd.concat([X_test[['form_soc', 'soc_len']], test_soc_signif], axis='columns')

X_train_description=pd.concat([X_train[['form_desc', 'desc_len']], description_text], axis='columns')
X_test_description=pd.concat([X_test[['form_desc', 'desc_len']], test_description_text], axis='columns')
del description_text
del test_description_text
del goal_text
del test_goal_text
del tasks_text
del test_tasks_text
del soc_signif
del test_soc_signif

del features_description
del test_features_description
del features_goal
del test_features_goal
del features_tasks
del test_features_tasks
del features_soc_signif
del test_features_soc_signif

Мы выбрали именно BernoulliNB, т.к. при переборе всех моделей эта показала лучший результат

In [83]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
bern_main=BernoulliNB()
bern_main.fit(X_train, y_train)
print("Бернулли_main: ", accuracy_score(bern_main.predict(X_test)>0.5, y_test))
bern_goal=BernoulliNB()
bern_goal.fit(X_train_goal, y_train)
print("Бернулли_goal: ", accuracy_score(bern_goal.predict(X_test_goal)>0.5, y_test))
bern_desc=BernoulliNB()
bern_desc.fit(X_train_description, y_train)
print("Бернулли_desc: ", accuracy_score(bern_desc.predict(X_test_description)>0.5, y_test))
bern_tasks=BernoulliNB()
bern_tasks.fit(X_train_tasks, y_train)
print("Бернулли_tasks: ", accuracy_score(bern_tasks.predict(X_test_tasks)>0.5, y_test))
bern_soc=BernoulliNB()
bern_soc.fit(X_train_soc, y_train)
print("Бернулли_soc: ", accuracy_score(bern_soc.predict(X_test_soc)>0.5, y_test))

Бернулли_main:  0.6876
Бернулли_goal:  0.7424
Бернулли_desc:  0.6784
Бернулли_tasks:  0.7168
Бернулли_soc:  0.6588


In [85]:
from sklearn.metrics import f1_score
print("Бернулли_main: ", f1_score(bern_main.predict(X_test)>0.5, y_test))
print("Бернулли_goal: ", f1_score(bern_goal.predict(X_test_goal)>0.5, y_test))
print("Бернулли_desc: ", f1_score(bern_desc.predict(X_test_description)>0.5, y_test))
print("Бернулли_tasks: ", f1_score(bern_tasks.predict(X_test_tasks)>0.5, y_test))
print("Бернулли_soc: ", f1_score(bern_soc.predict(X_test_soc)>0.5, y_test))

Бернулли_main:  0.3686337914308812
Бернулли_goal:  0.22033898305084745
Бернулли_desc:  0.36990595611285265
Бернулли_tasks:  0.24840764331210188
Бернулли_soc:  0.34334103156274054


In [118]:
from joblib import dump
dump(bern_main, 'bern_main.joblib')

['bern_main.joblib']